# Simple Stock Screener:

This is a simplified version of my original stock screener. My original stock screener has produced over 50% profit of my overall portfolio balance in a little over 4 months (Sept 2022 - Jan 2023). I have simplified this version to use a Golden Cross moving average strategy due to not wanting to share my trading strategy secrets. I have created this notebook with a few handy tools such as screening for stocks that fit a certain strategy (In this case, Golden Cross moving average), an alerter to alert when one or a list of stocks hit a specific price target, aswell as a backtesting function to backtest a strategy on a stock with specified timeframes, lookbacks, and starting balance.

In [ ]:
#Import all libraries as needed

import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
%pip install yfinance
import yfinance as yf
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import time
from warnings import filterwarnings
from pandas_datareader import data as pdr
%pip install ta
from ta.trend import MACD
#Graphing/Visualization
import datetime as dt 
import plotly
import plotly.graph_objs as go

%pip install yahoo_fin
from yahoo_fin import stock_info as si
import warnings

# Functions/Methods used:

### plotStock():
This function takes in parameters (Dataframe of the stock being plotted, macd being used from ta.trend, and stock which is just the name of the stock being plotted) it then uses plotly to plot the stock. The plot of the stock includes the macd indicator, 2 Moving averages - 200MA and 50MA, aswell as volume.

### scanStock():
This function uses the dataframe of the stock to check for certain criteria we are looking for in our strategy and marks whether or not the criteria are met with the booleans being used in the function. In this case we are looking for 4 things, if the stock has made a Golden Cross (50MA crosses 200MA), if the macd has crossed, if the macd is increasing, and whether or not the stock is currently undervalued (Based on the stocks bookvalue)

### checkStocks():
This is the main function being used in the stock screener. This function accepts 2 parameters, stockList: a list of a stock (or many stocks) which we are screening and strategy: The timeframe per candle in which we are checking for in our strategy (the 3 timeframes accepted are: 'd' for day, 'wk' for week, and 'mo' for month). In this function we use the yahoo finance library to get the prices of the stocks and book value and storing them into the dataframe so that they can be checked by scanStock() function. We also use the technical analysis library to store the macd values, which will also be checked in scanStock function. Lastly after we scan the stock with scanStock function and label the criteria met, we check if our strategy (Golden cross) is fulfilled. If it is, we plot the stock and this is how to know which stocks fit our strategy.


In [ ]:
warnings.filterwarnings("ignore")

def plotStock(df, macd, stock):
    # Declare plotly figure (go)
    fig = go.Figure()

    # add subplot properties when initializing fig variable
    fig = plotly.subplots.make_subplots(
        rows=3,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.01,
        row_heights=[0.5, 0.1, 0.2],
    )

    fig.add_trace(
        go.Candlestick(
            x=df.index,
            open=df["Open"],
            high=df["High"],
            low=df["Low"],
            close=df["Close"],
            name="market data",
        )
    )

    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df["MA50"],
            opacity=0.7,
            line=dict(color="blue", width=2),
            name="MA 50",
        )
    )

    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df["MA200"],
            opacity=0.7,
            line=dict(color="orange", width=2),
            name="MA 200",
        )
    )

    # Plot volume trace on 2nd row
    colors = [
        "red" if row["Open"] - row["Close"] >= 0 else "green"
        for index, row in df.iterrows()
    ]
    fig.add_trace(go.Bar(x=df.index, y=df["Volume"], marker_color=colors), row=2, col=1)

    # Plot MACD trace on 3rd row
    colorsM = ["green" if val >= 0 else "red" for val in macd.macd_diff()]
    fig.add_trace(
        go.Bar(x=df.index, y=macd.macd_diff(), marker_color=colorsM), row=3, col=1
    )
    fig.add_trace(
        go.Scatter(x=df.index, y=macd.macd(), line=dict(color="black", width=2)),
        row=3,
        col=1,
    )
    fig.add_trace(
        go.Scatter(x=df.index, y=macd.macd_signal(), line=dict(color="blue", width=1)),
        row=3,
        col=1,
    )

    # update layout by changing the plot size, hiding legends & rangeslider, and removing gaps between dates
    fig.update_layout(
        height=900, width=1200, showlegend=False, xaxis_rangeslider_visible=False
    )

    # Make the title dynamic to reflect whichever stock we are analyzing
    fig.update_layout(
        title=str(stock) + " Live Share Price:",
        yaxis_title="Stock Price (USD per Shares)",
    )

    # update y-axis label
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)
    fig.update_yaxes(title_text="MACD", showgrid=False, row=3, col=1)

    #Add slider (Not used in project)
    fig.update_xaxes(
        #Change to True if you want to add time slider.
        rangeslider_visible=False,
        rangeselector_visible=False,
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=15, label="15m", step="minute", stepmode="backward"),
                    dict(count=45, label="45m", step="minute", stepmode="backward"),
                    dict(count=1, label="HTD", step="hour", stepmode="todate"),
                    dict(count=1, label="1h", step="hour", stepmode="backward"),
                    dict(count=4, label="4h", step="hour", stepmode="backward"),
                    dict(count=1, label="1d", step="day", stepmode="backward"),
                    dict(count=7, label="1wk", step="day", stepmode="backward"),
                    dict(count=30, label="1mo", step="day", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
    )
    fig.show()
    time.sleep(5)
    plt.pause(0.005)


def scanStock(df):
    #Score and criteria in which we are looking for.
    score = 0
    undervalued = False
    macdCross = False
    macdIncreasing = False
    goldenCross = False
    
    #Check for our criteria and mark them true if they're met.
    if len(df) >= 220 and "MACD" in df:
        if (df["Close"][len(df) - 1] is not None and df["bookValue"][len(df) - 1] is not None):
            if df["Close"][len(df) - 1] < df["bookValue"][len(df) - 1]:
                undervalued = True
                score += 1
                print("Undervalued!")
        if(df["MA50"][len(df)-2] <= df["MA200"][len(df)-2] and df["MA50"][len(df)-1] > df["MA200"][len(df)-1] or df["MA50"][len(df)-3] <= df["MA200"][len(df)-3] and df["MA50"][len(df)-2] > df["MA200"][len(df)-2]):
            goldenCross = True
            score += 1
            print("Golden Cross Identified!")
        if df["MACD"][len(df) - 1] > df["MACD_sig"][len(df) - 1]:
            macdCross = True
            score += 1
            #print("MACD CROSSED")
        if(df["MACD"][len(df) - 1] - df["MACD_sig"][len(df) - 1] > df["MACD"][len(df) - 2] - df["MACD_sig"][len(df) - 2]):
            macdIncreasing = True
            score += 1
            #print("MACD INCREASING")
    
    #Return our score and criterias
    return score, goldenCross, macdCross, macdIncreasing, undervalued


def checkStocks(stockList, strategy):
    # Override Yahoo Finance
    yf.pdr_override()

    for i in range(len(stockList)):
        #Booleans for the different criteria we're checking for.
        macdCross = False
        macdIncreasing = False
        undervalued = False
        goldenCross = False
        
        #Get the current stock in the stockList we're using
        stock = stockList[i]
        print("Current Stock: ", stock)

        #Check for marketCap
        try:
            if(yf.Ticker(stock) is not None):
                thisMarketCap = yf.Ticker(stock).info['marketCap']
                print("Market Cap: ", thisMarketCap)
            else:
                thisMarketCap = 0
        except KeyError:
            print("Market Cap Couldn't be found for stock: ", stock)
            thisMarketCap = 0

        #Get different data based on which strategy we used
        if(strategy == 'd'):
            interval = '1' + strategy
            df = yf.download(tickers=stock, period="2y", interval=interval)
        elif(strategy == 'wk'):
            interval = '1' + strategy
            df = yf.download(tickers=stock, period="6y", interval=interval)
        elif(strategy == 'mo'):
            interval = '1' + strategy
            df = yf.download(tickers=stock, period="25y", interval=interval)
        
        #Check if the length of our dataframe is over 220, as we need atleast 200 for the 200 MA
        if len(df) >= 220:
            # MACD
            macd = MACD(close=df["Close"], window_slow=26, window_fast=12, window_sign=9)
            df["MACD"] = macd.macd()
            df["MACD_diff"] = macd.macd_diff()
            df["MACD_sig"] = macd.macd_signal()
            df["MA50"] = df["Close"].rolling(window=50).mean()
            df["MA200"] = df["Close"].rolling(window=200).mean()
            
            #Check if bookValue is available, if it is, store it, otherwise set default as 0.
            try:
                df["bookValue"] = yf.Ticker(stock).info["bookValue"]
            except KeyError:
                df["bookValue"] = 0 

            #Mark which criteria are met with scanStock function.
            (thisStockScore, goldenCross, macdCross, macdIncreasing, undervalued) = scanStock(df)
            
                
        #Using the marked criteria from above, now we check for the proper trading pattern we're looking for
        if(goldenCross):
            print("This stock scored: ", thisStockScore)
            plotStock(df, macd, stock)

### Get list of stocks and screen for Golden Cross

In this section we use the yahoo_fin library to get the ticker symbols from all of the major US exchanges (S&P 500, Nasdaq, Dow, and Others) and use the lists to put into our screening function (checkStock).

In [ ]:
# gather stock symbols from major US exchanges
df1 = pd.DataFrame( si.tickers_sp500() )
df2 = pd.DataFrame( si.tickers_nasdaq() )
df3 = pd.DataFrame( si.tickers_dow() )
df4 = pd.DataFrame( si.tickers_other() )

df1.reset_index

# convert DataFrame to list, then to sets
sym1 = set( symbol for symbol in df1[0].values.tolist() )
sym2 = set( symbol for symbol in df2[0].values.tolist() )
sym3 = set( symbol for symbol in df3[0].values.tolist() )
sym4 = set( symbol for symbol in df4[0].values.tolist() )

# join the 4 sets into one. Because it's a set, there will be no duplicate symbols
symbols = set.union( sym1, sym2, sym3, sym4 )

# Some stocks are 5 characters. Those stocks with the suffixes listed below are not of interest.
my_list = ['W', 'R', 'P', 'Q']
del_set = set()
sav_set = set()

for symbol in symbols:
    if len( symbol ) > 4 and symbol[-1] in my_list:
        del_set.add( symbol )
    else:
        sav_set.add( symbol )

print( f'Removed {len( del_set )} unqualified stock symbols...' )
print( f'There are {len( sav_set )} qualified stock symbols...' )

#Enter a list of stocks to check in presetList and call checkStocks on it to check a set of chosen stocks
presetList=[]

checkStocks(presetList)
#Check the dataframes of the stocks that were found in the different market exchanges (s&p, dow, nasdaq, etc..)
checkStocks(df1[0], 'd')
checkStocks(df2[0], 'wk')
checkStocks(df3[0], 'wk')



## Misc/Utilities:

### setAlertForPriceLevels():
In this function we pass in a list of stocks as well as a list of price levels which correspond to the order of the stock list. For example, we want to watch if TSLA hits a price of 120 and NFLX hits a price of $350, we would call the function as such: setAlertForPriceLevels(["TSLA","NFLX"], [120,350]). If any of the stocks reaches the levels we're looking for, it will alert via an audio queue (audio.wav) to let you know that your price level has been hit.

### backTestStock():
This function is used to backtest a strategy with 5 inputs: the stock we are backtesting on, an amount of starting cash we'd like to start with, timeframe per candle, lookBackInYears (How far back on the stock we'd like to backtest), and the strategy (In this case we have 2 different strategies we can backtest with - "M" which buys when the macd signal crosses over the macd line and sells when the macd signal crosses under the macd line again AND "G" for Golden Cross strategy which buys when the 50MA Crosses the 200MA and sells when the 200MA crosses back under the 50MA). After this function finishes backtesting on the stock, it will give the amount of cash in which it ended at aswell as a ratio, how many times the strategy gained profit (win) and lost money (loss).

In [ ]:
from IPython.display import Audio 
from IPython.core.display import display

def setAlertForPriceLevel(stocks, priceLevels):
    i = 0
    levelPassed = False
    while(levelPassed == False):
        for i in range(0,len(stocks)):
            df = yf.download(tickers=stocks[i],period='1y',interval='1mo')
            getInfo = yf.Ticker(stocks[i])
            percentChangeNeeded = round(((priceLevels[i] - df['Close'][len(df)-1]) / df['Close'][len(df)-1]) * 100, 2)
            print("Checking Current Stock: ", stocks[i], " Target Price: ", priceLevels[i], " Current Price: ", round(df['Close'][len(df)-1], 3), " Percent Change Needed: ", percentChangeNeeded,'%')
            if(df['Close'][len(df)-1] >= priceLevels[i]):
                for i in range(0,100):
                    display(Audio('alert.wav', autoplay=True))
                    break;
            time.sleep(3)
          
        

def backTestStock(stock, startingCash, timeFrame, lookBackInYears, strategy):
    print("Testing: ", stock)
    startingCash = startingCash
    lastCash = 0
    inAposition = False
    wins = 0
    losses = 0

    interval = "1" + timeFrame
    period = str(lookBackInYears) + 'y'
    print(period)
    
    df = yf.download(tickers=stock,period=period,interval=interval)
    
    getInfo = yf.Ticker(stock)

    if(len(df) >= 220):
        # MACD
        macd = MACD(close=df['Close'], 
                      window_slow=26,
                      window_fast=12, 
                      window_sign=9)
        df["MA50"] = df["Close"].rolling(window=50).mean()
        df["MA200"] = df["Close"].rolling(window=200).mean()
        df['MACD'] = macd.macd()
        df['MACD_diff'] = macd.macd_diff()
        df['MACD_sig'] = macd.macd_signal()
        if('bookValue' in getInfo.info):
            df['bookValue'] = getInfo.info['bookValue']
        else:
            df['bookValue'] = 0

    if(strategy == 'm' or strategy == 'M'):
        for i in range(0,len(df)):
            if(inAposition == False and df['MACD'][i-1] > df['MACD_sig'][i-1] and df['MACD'][i-2] <= df['MACD_sig'][i-2]):
                currentPosition = startingCash / df['Close'][i-1]
                lastCash = startingCash
                print(df.index[i-1], "Started Position: ", startingCash)
                startingCash -= startingCash
                inAposition = True
            
            if(inAposition and df['MACD'][i-1] < df['MACD_sig'][i-1] and df['MACD'][i-2] >= df['MACD_sig'][i-2]):
                startingCash += currentPosition * df['Close'][i]
                currentPosition = 0
                inAposition = False
                print(df.index[i], "Ended Position: ", startingCash, " Last Cash : ", lastCash)
                if(lastCash < startingCash):
                    wins += 1
                else:
                    losses += 1
        print("Wins: ", wins, " Losses: ", losses)
    elif (strategy == 'g' or strategy == 'G'):
        for i in range(0,len(df)):
            if(inAposition == False and df['MA50'][len(df)-1] > df['MA200'][len(df)-1] and df['MA50'][len(df)-2] <= df['MA200'][len(df)-2]):
                currentPosition = startingCash / df['Close'][i-1]
                lastCash = startingCash
                startingCash -= startingCash
                inAposition = True
                if(inAposition):
                    print(df.index[i-1], "Started Position: ", startingCash)
                
            if(inAposition and df['MA50'][i-1] < df['MA200'][i-1] and df['MA50'][i-2] >= df['MA200'][i-2]):
                startingCash += currentPosition * df['Close'][i]
                currentPosition = 0
                inAposition = False
                print(df.index[i], "Ended Position: ", startingCash, " Last Cash : ", lastCash)
                if(lastCash < startingCash):
                    wins += 1
                else:
                    losses += 1
        print("Wins: ", wins, " Losses: ", losses)

In [ ]:
#Examples of the functions being used:
setAlertForPriceLevels(["TSLA","NFLX"], [120,350])
backTestStock("ALK", 10000, 'd', 10, 'G')